In [66]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd

2. Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [62]:
data = pd.read_csv("SMSSpamCollection.txt", sep='\t', header=None)
data.columns = ['target', 'text']
data.head()

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


3. Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [39]:
y = data.target.map({'spam': 1, 'ham': 0}).values
x = data.text.values

4. Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [40]:
vectorizer = CountVectorizer()
x = vectorizer.fit_transform(x)

5. Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [49]:
scores = cross_val_score(LogisticRegression(solver='lbfgs'), x, y, cv=10, scoring='f1')
round(np.mean(scores), 1)

0.9

6. А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:  
"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.



In [53]:
test = [ "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
"FreeMsg: Txt: claim your reward of 3 hours talk time",
"Have you visited the last lecture on physics?",
"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
"Only 99$"]

In [54]:
clf = LogisticRegression(solver='lbfgs')
clf.fit(x, y)
preds = clf.predict(vectorizer.transform(test))
preds

array([1, 1, 0, 0, 0])

7. Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [56]:
ngram_range = [(2,2), (3,3), (1,3)]
res = []
for n in ngram_range:
    x = data.text.values
    vectorizer = CountVectorizer(ngram_range=n)
    x = vectorizer.fit_transform(x)
    scores = cross_val_score(LogisticRegression(solver='lbfgs'), x, y, cv=10, scoring='f1')
    res.append(round(np.mean(scores), 2))
res

[0.82, 0.73, 0.92]

8. Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор.

In [72]:
#x = data.text.values
#vectorizer = CountVectorizer(ngram_range=(1,1))
#x = vectorizer.fit_transform(x)
#scores = cross_val_score(MultinomialNB(), x, y, cv=10, scoring='f1')
#round(np.mean(scores), 2)

ngram_range = [(2,2), (3,3), (1,3)]
res = []
for n in ngram_range:
    x = data.text.values
    vectorizer = CountVectorizer(ngram_range=n)
    x = vectorizer.fit_transform(x)
    scores = cross_val_score(MultinomialNB(), x, y, cv=10, scoring='f1')
    res.append(round(np.mean(scores), 2))
res

[0.65, 0.38, 0.89]

9. Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [65]:
x = data.text.values
vectorizer = TfidfVectorizer(ngram_range=(1,1))
x = vectorizer.fit_transform(x)
scores = cross_val_score(LogisticRegression(solver='lbfgs'), x, y, cv=10, scoring='f1')
round(np.mean(scores), 2)

0.85